In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

pd.set_option('display.max_columns', None)


In [ ]:
print("Please upload your CSV file:")
uploaded = files.upload()  # Open file picker
filename = list(uploaded.keys())[0]
data = pd.read_csv(filename)

# Quick check
data.head()

Please upload your CSV file:


In [ ]:
num_records = data.shape[0]
num_income_gt_50k = (data['income'] == '>50K').sum()
num_income_le_50k = (data['income'] == '<=50K').sum()
pct_income_gt_50k = (num_income_gt_50k / num_records) * 100

print("Number of records:", num_records)
print("Number of individuals with income >50K:", num_income_gt_50k)
print("Number of individuals with income <=50K:", num_income_le_50k)
print("Percentage of individuals with income >50K: {:.2f}%".format(pct_income_gt_50k))


In [ ]:
X = data.drop('income', axis=1)
y = data['income']

categorical_features = X.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse=False, drop='first')
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_features]), columns=encoder.get_feature_names_out(categorical_features))

X_numeric = X.select_dtypes(include=[np.number])
X_preprocessed = pd.concat([X_numeric.reset_index(drop=True), X_encoded.reset_index(drop=True)], axis=1)

y_encoded = y.map({'>50K': 1, '<=50K': 0})

X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y_encoded, test_size=0.2, random_state=42)


In [ ]:
y_naive_pred = np.ones_like(y_test)
accuracy_naive = accuracy_score(y_test, y_naive_pred)
f1_naive = f1_score(y_test, y_naive_pred)

print("Naive Predictor Accuracy: {:.4f}".format(accuracy_naive))
print("Naive Predictor F1 Score: {:.4f}".format(f1_naive))


In [ ]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(),
    'AdaBoost': AdaBoostClassifier()
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    results[name] = {'Accuracy': acc, 'F1 Score': f1}

results_df = pd.DataFrame(results).T
print(results_df)

# Plot model performance
results_df.plot(kind='bar', figsize=(10,6))
plt.title('Model Performance Comparison')
plt.ylabel('Score')
plt.xticks(rotation=0)
plt.show()


In [ ]:
param_grid = {'max_depth': [5, 10, 15]}
grid = GridSearchCV(RandomForestClassifier(), param_grid, scoring='f1', cv=5)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_
y_pred_best = best_model.predict(X_test)

print("Best parameters:", grid.best_params_)
print("Optimized Model Accuracy:", accuracy_score(y_test, y_pred_best))
print("Optimized Model F1 Score:", f1_score(y_test, y_pred_best))


In [ ]:
importances = best_model.feature_importances_
feature_names = X_preprocessed.columns
feat_imp_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances}).sort_values(by='Importance', ascending=False)

top5_features = feat_imp_df.head(5)
print(top5_features)

sns.barplot(x='Importance', y='Feature', data=top5_features)
plt.title('Top 5 Feature Importances')
plt.show()


In [ ]:
X_train_top5 = X_train[top5_features['Feature']]
X_test_top5 = X_test[top5_features['Feature']]

best_model.fit(X_train_top5, y_train)
y_pred_top5 = best_model.predict(X_test_top5)

print("Top 5 Features Model Accuracy:", accuracy_score(y_test, y_pred_top5))
print("Top 5 Features Model F1 Score:", f1_score(y_test, y_pred_top5))


In [ ]:
### Recommendation Based on Model Results

Based on the evaluation of our models, I recommend **building an in-house solution** if moderate accuracy is sufficient, as our Random Forest model with optimized hyperparameters achieved a strong F1 score and accuracy. Compared to a naive predictor or a simple Logistic Regression model, our tuned Random Forest performs significantly better and captures nonlinear relationships in the data.

Although our model does not match the performance of advanced pre-trained or commercial solutions, it is interpretable, customizable, and efficient for moderate-scale projects. To further improve accuracy, additional steps could include feature engineering, ensemble methods, or using larger datasets. Overall, for most practical scenarios, building this solution in-house is feasible and cost-effective, while purchasing or using a pre-trained solution may only be necessary for high-stakes or large-scale deployment.